# BWT READ MATCHING ALGORITHM
implemented BWT read matching algorithm using class having private data members as the data structures required for the alog<br>
and member function as the untill functionality required for the algo<br>
this algo works for any other string as well not just for genomic sequence

In [3]:
class BWT_Read_Matching:

    # a class for implementing the whole BWT read matching algorithm
    # having data members as the required data structures for the algo
    # and all the functionalltiy(rank, select, find_patter..e.tc) as the member function of the class


    n = 0            #length of BWT string
    idx = []         #compressed index
    cnt = {}         #cnt[i]: count of char i
    rank = {}        #rank: containing rank of every delta_1 element
    delta_1 = 0      #for optimizing rank space
    delta_2 = 0      #for optimizing index space
    s_chars = []     #list of all chars of BWT string
    bit_array = {}   #bit_array[ch][i] = 1 if BWT_string[i] == ch otherwise 0

    def __init__(self, s, id, delta_1, delta_2):
        # constructor intialization performs the following pre-computation
        # populating the bit array
        # creating the rank matrix
        # find count of chars in the input string
        # compressing the list of indices

        # Parameters:
        # s: last col of bwt of input string
        # id: map array of bwt of input string
        # delta_1: milestone for optimizing the rank opperation space complexity
        # delta_2: milestone for optimzing the find index operation space complexity 

        self.delta_1 = delta_1
        self.delta_2 = delta_2

        self.n = len(s)
        N = self.n//self.delta_1

        self.s_chars = self.get_char(s)
        self.populate_bit_array(s)     #populating the bit array

        for char in self.s_chars:     #initializing the rank matrix
            a = [0] * N if self.n % delta_1 == 0 else [0] * (N + 1)
            self.rank.setdefault(char, a)

        for char in self.rank:       #populating rank matrix w.r.t delta_1

            cnt = 0
            for i in range(N):

                start = i * delta_1
                end = (i + 1)*delta_1

                for j in range(start, end):
                    if self.bit_array[char][j] == "1":
                        cnt += 1

                self.rank[char][i] = cnt

            for j in range(N * delta_1, self.n):
                if self.bit_array[char][j] == "1":
                    cnt += 1

                self.rank[char][self.n//delta_1] = cnt

        self.cnt = self.cnt_freq(s)  #finding count of all characters in BWT_string
        self.compress_idx(id, delta_2) #compressing list of index

    def find_rank(self, char, pos):
        # find the rank of char at the position pos 

        # Parameters:
        # char: char who's rank is to be found
        # pos: postion at which the rank of char is to be found

        # Returns:
        # rank of char at pos 

        if char not in self.s_chars: 
            return 1

        r = 0
        idx = pos - 1
        if pos % self.delta_1 == 0:

            r = self.rank[char][(pos//self.delta_1) - 1]
            if self.bit_array[char][idx] == "1":
                r -= 1

        else:
            lb = (pos//self.delta_1) * self.delta_1 #creating the upper and lower bound in which we have to iterate
            ub = min(self.n, lb + self.delta_1)

            if idx - lb <= ub - idx: #finding the closest bound

                r = self.rank[char][(pos//self.delta_1) - 1] if pos >= self.delta_1 else 0

                for i in range(lb, idx):
                    if self.bit_array[char][i] == "1":
                        r += 1
            else:
                r = self.rank[char][pos//self.delta_1]

                for i in range(idx, ub):
                    if self.bit_array[char][i] == "1":
                        r -= 1

        return r

    def populate_bit_array(self, s):
        # populates the bit array for all char of the bwt input string

        # Parameters:
        # s: input bwt string

        n = len(s)

        for char in self.s_chars:

            self.bit_array.setdefault(char, "")

            for i in range(n):
                if s[i] == char:
                    self.bit_array[char] += "1"
                else:
                    self.bit_array[char] += "0"

    def get_char(self, s): 
        # creats the sorted list of charachters in BWT_string

        # Parameters:
        # s: input bwt string

        # Returns:
        # sorted list of chars in input bwt string

        d = []
        for i in range(len(s)):
            if s[i] not in d:
                d.append(s[i])

        d.remove("$")
        d.sort()
        return d

    def cnt_freq(self, s): 
        # finds frequency of all charachters in BWT_string

        # Parameters:
        # s: input bwt string

        # Returns:
        # dictionary containing the count of each chars in input bwt string

        cnt = {}
        for char in self.s_chars:
            cnt.setdefault(char, 0)

        for i in range(len(s)):
            if s[i] != "$":
                cnt[s[i]] += 1

        return cnt

    def compress_idx(self, id, delta_2): 
        # compress the list of indices(id) w.r.t delta_2

        # Parameters:
        # id: map array of bwt of input string
        # delta_2: milestone for optimizing find index space complexity

        i = delta_2

        while i <= self.n:
            self.idx.append(id[i - 1])
            i += delta_2

    def select(self, char, rank): 
        # finds the position of char having rank: rank in first col of bwt of input string

        # Parameters:
        # char: char of the bwt string
        # rank: rank of the char, who's position we have to find

        # Returns:
        # position of char having rank: rank, in the first col of bwt of input string

        pos = 1
        if char == "$":
            return pos

        for ch in self.s_chars:
            if ch == char:
                if rank == self.cnt[ch]:  #if rank == cnt[char] return the position of last occurence of char
                    pos += rank
                else:
                    pos += (rank + 1)
                break
            else:
                pos += self.cnt[ch]
        pos = min(pos, self.n)
        return pos

    def find_pattern_utill(self, l, h, s, i):
        # recursively search in the band [l, h] for the ith char of 
        # input pathern s, and finally returns the position at which pattern s matches

        # Parameters:
        # l: first band position (1 based index)
        # h: last band position (1 based index)
        # s: pattern which we are searching
        # i: current position of the char which we are searching in the band [l,h] (0 based index)

        # Returns:
        # all the position(1 based index) at which pattern s matches in the input string
        # [-1] if the pattern s doesn't matches anywhere

        if i < 0:  #pattern is exhauhsted

            pos = []
            for i in range(l, h + 1): #simply returning the index of the final band
                pos.append(self.find_index(i, 0))

            pos.sort()
            return pos

        r1 = self.find_rank(s[i], l) 
        r2 = self.find_rank(s[i], h)

        if r1 == r2 and (s[i] not in self.s_chars or self.bit_array[s[i]][h - 1] != "1"): #if rank in last and first row are equal check if the last row last char is equal to s[i] or s[i] belongs to the list of chars of BWT string 
            return [-1]

        else:
            if self.bit_array[s[i]][h - 1] != "1": #if last char at the last band(h) is not equal to the current char
                r2 -= 1

        l = self.select(s[i], r1)
        h = self.select(s[i], r2)

        return self.find_pattern_utill(l, h, s, i - 1)

    def find_pattern(self, s):
        # finds all position where pattern s matches in the input string

        # Parameters:
        # s: input pattern

        # Returns:
        # all the position where s matches in the input string, [-1] if doesn't matches anywhere

        return self.find_pattern_utill(1, self.n, s, len(s) - 1)

    def find_index(self, i, pos):
        # recursively finds the index for the ith row in the bwt of input string
        # using compress index list

        # Parameters:
        # i: position of row in the bwt of input string(1 based index)
        # pos: offeset that we have to add finally after getting a hit in compressed idx array

        # Returns:
        # the position in the orignall string of the first char of ith row in the bwt of input string

        if (i % self.delta_2) == 0:  #if row number is divisible by delta_2

            return (self.idx[(i//self.delta_2) - 1] + pos)%self.n

        ch = "" #last char in row i
        for char in self.s_chars:
            if self.bit_array[char][i - 1] == "1":
                ch = char
                break

        if ch == "":
            ch = "$"

        r = self.find_rank(ch, i)
        i = self.select(ch, r)

        return self.find_index(i, pos + 1) #increase pos by 1 from the starting idx

    def find_pattern_approx(self, s, p, k): 
        # finds all the position where p matches in string s with at-most k miss-matches

        # s: input string
        # p: pattern
        # k: allowed number of miss-matches

        # Returns:
        # all the position where pattern(p) matches in the string(s) with at most k miss-mathes

        n = len(p)
        N = len(s)
        if k >= n:
            pos = list(range(1, self.n - n))
            return pos

        else:

            K = k + 1 #divide the pattern in k + 1 parts
            l = n//K  #minimum length of each part

            part = ""
            L = R = 0
            match_pos = []

            for i in range(K):

                L = i*l - 1 #starting index for iterating in the left part
                if i == K - 1:
                    R, part = n, p[i*l: n]
                else:
                    R, part = (i + 1)*l, p[i*l: (i + 1)*l] #starting index for iterating in the right part

                pos = self.find_pattern(part) 
                l = len(part)

                if pos[0] == -1:
                    continue

                for idx in pos: #for each pos of the current part iteratively check for the left & right part of the pattern accounting at max k miss-matches
                    if (idx - 1  >= L + 1) and (N - (idx + l) + 1 >= n - R) and((idx - (L + 1)) not in match_pos):  #1st: left part of p < left part of s, 
                                                                                                                    #2nd: right part of p < right part of s,
                                                                                                                    #3rd current part, idx dosen't matches to already existing matching postion

                        p1, cnt, p2 = L, 0, idx - 2

                        while p1 >= 0 and cnt <= k:

                            if p[p1] != s[p2]:
                                cnt += 1

                            p1 -= 1
                            p2 -= 1

                        if cnt > k:
                            continue

                        p1, p2 = R, idx + l - 1

                        while p1 < n and cnt <= k:

                            if p[p1] != s[p2]:
                                cnt += 1

                            p1 += 1
                            p2 += 1

                        if cnt > k:
                            continue

                        match_pos.append(idx - (L + 1))
            match_pos.sort()
            return match_pos if len(match_pos) else [-1]

In [2]:
def BWT(s):
    # finds the index map and last column of bwt of a string

    # Parameter:
    # s: input string

    # Returns:
    # a list containing the index map and last col colmn of bwt of s

    s = "$" + s
    n = len(s)
    l = []

    for i in range(0, n):
        s = s[1:] + s[0]
        l.append([s, (i + 1) % n])

    l.sort()

    s = ""
    idx = []
    for i in range(len(l)):
        s += l[i][0][-1]
        idx.append(l[i][1])

    return [s, idx]

# HOW TO USE THE CLASS
Example of how to declare the BWT_Read_Matcing class object & use it's various functions

In [3]:
# s = "ACGTTTGATCG"
# bw = BWT(s)
# print(f"last_col: {bw[0]}")
# print(f"list of indices: {bw[1]}")
# brm = BWT_Read_Matching(bw[0], bw[1], 2, 5)

In [4]:
# print(f"n: {brm.n}")
# print(f"delta_1: {brm.delta_1}")
# print(f"delta_2: {brm.delta_2}")
# print(f"s_chars: {brm.s_chars}")
# print(f"cnt: {brm.cnt}")
# print(f"rank: {brm.rank}")
# print(f"bit_array: {brm.bit_array}")
# print(f"idx: {brm.idx}")

# brm.find_pattern("AN")

In [5]:
# brm.find_pattern_approx(s, "ACC", 1)